# 1. Extraindo dados do Spotify

## a) Importando as bibliotecas

Obs.: Spotipy é uma biblioteca desenvolvida para ler as APIs do Spotify (https://spotipy.readthedocs.io/en/latest/#)

In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
from json.decoder import JSONDecodeError

## b) informar seus IDs de developer Spotify

In [2]:
cid = 'seu_client_id' #client id
sid = 'seu_secret_id' #secret id

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=sid)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

## c) Buscar as faixas disponíveis no Brasil por ano de lançamento
Esta busca retornará as músicas, com seu ano de lançamento.

A popularidade de cada faixa é um número de 0 a 100, sendo 100 mais popular e 0 menos popular.

É a popularidade atual da faixa, de acordo com as execuções mais recentes dos usuários no Spotify.

Como há um limite de 2000 faixas a cada chamada, optei por segregar por ano, assim consigo uma amostra muito maior. Como as faixas mais populares aparecem primeiro, e posteriormente vou me concentrar nas faixas + populares, não é um problema não conseguir os dados totais, que devem ser bem mais numerosos. 

In [4]:
# Criar listas para guardar os dados
artist_name = []
track_name = []
track_id = []
track_preview_url = []
track_popularity = []
track_year = []
album = []
album_art = []
duration_ms = []
release_date = []

# Percorrer os anos de 1960 até 2021
initial_year = 1960
final_year = 2021

for ano in range(initial_year, (final_year+1)):
    print(ano)
    for i in range(0,2000,50):
        track_results = sp.search(q='year:{}'.format(ano), type='track', limit=50,offset=i, market='BR')
        for i, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            track_preview_url.append(t['preview_url'])
            track_popularity.append(t['popularity'])
            track_year.append(ano)
            album.append(t['album']['name'])
            try:
                album_art.append(t['album']['images'][1]['url']) #há 3 figuras. A primeira c/ maior resolucao, peguei a primeira.
            except Exception:
                album_art.append('not found')
            duration_ms.append(t['duration_ms'])
            release_date.append(t['album']['release_date'])


1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


## d) Guardar os resultados em um DataFrame

In [6]:
pd.options.display.max_colwidth = 100

track_dataframe = pd.DataFrame({'artist_name' : artist_name,
                                'track_name' : track_name,
                                'album' : album,
                                'track_popularity' : track_popularity,
                                'track_id' : track_id,
                                'track_preview_url' : track_preview_url,
                                'album_art' : album_art,
                                'duration_ms' : duration_ms,
                                'track_year' : track_year,
                                'release_date' : release_date})

print(track_dataframe.shape)


(122200, 10)


## e) Eliminar eventuais músicas duplicadas

In [7]:
# eliminar track_id's duplicados
track_dataframe = track_dataframe.drop_duplicates(subset="track_id")
print(track_dataframe.shape)

(122200, 10)


## f) Salvar em um arquivo csv

In [8]:
from datetime import date
hoje = date.today().strftime("%Y-%m-%d")

track_dataframe.to_csv("spotify_tracks_{}.csv".format(hoje), quotechar='"', escapechar='"')

In [9]:
track_dataframe.sample(5)

,artist_name,track_name,album,track_popularity,track_id,track_preview_url,album_art,duration_ms,track_year,release_date
67039,Milton Banana Trio,Flor de Lis,Sambas De Bossa,21,6jx9BuEQdv5YQIw94lbnJ9,https://p.scdn.co/mp3-preview/c5cbf3aba736ac46164b019e1dd35eb1c2a1ae3d?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e028cf384d2538a93c6b77332c7,219773,1993,1993-07-15
26316,Vieira & Vieirinha,Peão De Boiadeiro,Peão De Boiadeiro,27,76vMPuNcYwOOiuXu8AdsOb,None,https://i.scdn.co/image/ab67616d00001e02071cd9e424f784c20320ab17,176866,1973,1973-02-01
35904,Anna Jantar,Jak w taki dzień deszczowy,Zawsze gdzieś czeka ktoś,22,6jqMisUjaOX9labGSDO4ev,https://p.scdn.co/mp3-preview/13732ef50e9afff43c4a908abf48b313be870acd?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e02f0bd466ef8ce5cd8f4921572,182360,1977,1977-02-12
69087,S. P. Balasubrahmanyam,Andhama Andhama,Govinda Govinda,38,6wfvttks8elZeW0h1IeAfp,https://p.scdn.co/mp3-preview/315260d713bf1a7f349b71e3ec590510aaa18518?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e0286087138af5e98d1b46ee071,320266,1994,1994-01-21
90688,Eduardo Costa,Página de Amígps,No Boteco,36,2UTJXcdSuX89pu94GSuRz9,https://p.scdn.co/mp3-preview/ed1db98aaaee8c5189277a78ff27b0ffb933bce0?cid=d779d6da93d24de483843...,https://i.scdn.co/image/ab67616d00001e020f83a80f377edc695147b5b5,203476,2005,2005
